# Running InstructLab with a GPU

<ul>
<li>Contributors: InstructLab team and IBM Research Technology Education team
<li>Contact for questions and technical support: IBM.Research.JupyterLab@ibm.com
<li>Provenance: IBM Research
<li>Version: 1.0.9
<li>Release date: 2024-2-7
<li>Compute requirements: CPU
<li>Memory requirements: 16 GB
<li>Notebook set: InstructLab
</ul>

## Summary
This notebook set demonstrates InstructLab, an open source AI project that facilitates knowledge and skills contributions to Large Language Models (LLMs). InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081). The open source InstructLab repository is available [here](https://github.com/instructlab/instructlab) and provides additional documentation on using InstructLab.

<img src="https://github.com/KenOcheltree/ilab-colab/blob/main/data/images/Flow.png?raw=1" width="1000">

InstructLab  can be instantiated in several different forms, depending on the processing capabilities available. InstructLab can take the form of an open source installation or a Red Hat AI InstructLab installation. The open source installation can be run on a range of hardware from a laptop to a build your own (BYO) server instance running on a Virtual Machine (VM). The below figure shows the different available instantiations of InstructLab.

<img src="https://github.com/KenOcheltree/ilab-colab/blob/main/data/images/experiences.png?raw=1" width="1000">

In this notebook set, we will be demonstrating both the open source version running on a VM server and Red Hat Enterprise Linux AI InstructLab running on an IBM Cloud Server.

The open source version running on a server is demonstrated in the following sections that are run sequentially:
* [Configuring InstructLab](#IL1_0)
* [Training with InstructLab](#IL2_0)
* [Inferencing with InstructLab](#IL3_0)

The Red Hat Enterprise Linux AI InstructLab is demonstrated running as a service in the IBM Cloud by running the following notebooks:
- Configuring InstructLab
- Training with Red Hat AI InstructLab Service
- Inferencing with Redhat-AI-InstructLab Trained Model
**Note:** The **Configuring InstructLab** notebook is run before the other Red Hat AI InstructLab notebooks to ensure that the *granite 7b model* is installed for inference comparisons as the base model.

<a id="IL0_0"></a>
# 1. Configuring InstructLab

This notebook demonstrates the configuration of InstructLab. The InstructLab method consists of three major components:
* **Taxonomy-driven data curation:**  The taxonomy is a set of training data curated by humans as examples of new knowledge and skills for the model.
* **Large-scale synthetic data generation:** A teacher model is used to generate new examples based on the seed training data. Recognizing that synthetic data can vary in quality, the InstructLab method adds an automated step to refine the example answers, ensuring they are grounded and safe.
* **Iterative model alignment tuning:** The model is retrained based on the synthetic data. The InstructLab method includes two tuning phases: knowledge tuning, followed by skill tuning.

In this notebook, we will demonstrate the following steps:

* [Step 0. Environment Preconfiguration](#IL1_preconfig)
* [Step 1. Check the Starting Configuration](#IL1_init)
* [Step 2. Configure InstructLab](#IL1_config)
* [Step 3. Download Models](#IL1_down)

**Note:** This notebook must be run within a GPU session. If you are not running with a GPU, please select **File->Hub Control Panel->Stop My Server**, then **Start My Server** and then select a GPU Session.

<a id="IL1_preconfig"></a>
## Step 1.0 Environment Preconfiguration
This step has already been performed for this JupyterLab environment and requires no work by the user. This information is provided in case the user is setting up their own environment on their own laptop or server.

The full steps for a direct installation are [here](https://github.com/instructlab/instructlab).



In [1]:
!python --version
!git clone https://github.com/KenOcheltree/ilab.git

Python 3.11.11
Cloning into 'ilab'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 55 (delta 13), reused 37 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 1.01 MiB | 3.15 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
!pip cache remove llama_cpp_python

Files removed: 0


In [ ]:
#!pip install rouge-score

In [ ]:
#!pip install deepspeed

In [ ]:
#!pip install flash-attn

In [ ]:
#!pip install sqlitedict

In [ ]:
#!pip install word2number

In [3]:
!pip install vllm==0.6.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [ ]:
!pip install llama_cpp_python[server]==0.3.2 -C cmake.args='-DLLAMA_CUDA=on'

In [ ]:
!pip install instructlab==0.24.0

### Check InstructLab Version

Check that InstructLab version 0.24.0 is installed properly and is configured for using a GPU.

The first line from 'InstructLab' section should read
```
instructlab.version: 0.24.0
```

In [ ]:
!ilab system info

<a id="IL1_check"></a>
## Step 1.1 Check for a GPU

This code cell checks for a GPU in the configuration. This notebook requires a GPU in the configuration to run properly.

In [ ]:
## standard imports
import os
import torch
from IPython.display import Image, display
import ipywidgets as widgets

H1 = "<p style='font-family:IBM Plex Sans;font-size:28px'>"
H2 = "<p style='font-family:IBM Plex Sans;font-size:24px'>"
Norm = "<p style='font-family:IBM Plex Sans;font-size:20px'>"
Small = "<p style='font-family:IBM Plex Sans;font-size:17px'>"
Ex = "<p style='font-family:IBM Plex Sans;font-size:20px;font-style:italic'>"

## torch and cuda version check
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

if torch.cuda.is_available() is False:
    print("No GPU in configuration")
else:
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    print("GPU is Available")

<a id="IL1_config"></a>
## Step 1.2 Configure InstructLab

### 1.2.1 Create InstructLab config file
The InstructLab configuration is captured in the *config.yaml* file. This step creates the config.yaml file and sets:
- **taxomony_path = taxonomy** - the root location of the taxonomy is set to the taxonomy folder in instructlab-latest
- **model_path = models/merlinite-7b-lab-Q4_K_M.gguf** - the default model is set to merlinite

**Note:** The default directories for InstructLab are the following. If you initialize InstructLab on your own system, it will default to the following:
* **Downloaded Models:**  ~/.cache/instructlab/models/ - Contains all downloaded large language models, including the saved output of ones you generate with ilab.
* **Synthetic Data:** ~/.local/share/instructlab/datasets/ - Contains data output from the SDG phase, built on modifications to the taxonomy repository.
* **Taxonomy:** ~/.local/share/instructlab/taxonomy/ - Contains the skill and knowledge data.
* **Training Output:** ~/.local/share/instructlab/checkpoints/ - Contains the output of the training process.
* **config.yaml:** ~/.config/instructlab/config.yaml - Contains the config.yaml file

In [ ]:
import shutil
if os.path.exists("sample_data"):
    print("removing sample_data")
    shutil.rmtree("sample_data")
os.chdir("ilab")

In [ ]:
base_dir="/root/"
##Choose the base model as granite or mixtral
model_dir="models"
model_name="granite-7b-lab-Q4_K_M.gguf"
model_path = os.path.join(model_dir, model_name)

taxonomy_path='taxonomy'

## Define the file name
file_name = "config.yaml"
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"ilab was already initialized. {file_name} has been deleted. Reinitialized")
else:
    print(f"ilab was not initialized yet. {file_name} does not exist.")

##Remove old data
if os.path.exists("taxonomy"):
    print("removing taxonomy")
    shutil.rmtree("taxonomy")
if os.path.exists(base_dir+".cache/instructlab"):
    print("removing " + base_dir+".cache/instructlab")
    shutil.rmtree(base_dir+".cache/instructlab")
if os.path.exists(base_dir+".config/instructlab"):
    print("removing " + base_dir+".config/instructlab")
    shutil.rmtree(base_dir+".config/instructlab")
if os.path.exists(base_dir+".local/share/instructlab"):
    print("removing " + base_dir+".local/share/instructlab")
    shutil.rmtree(base_dir+".local/share/instructlab")

print(f"ilab model is {model_path}.")
print('#############################################################')
print(' ')

command = f"""
ilab config init<<EOF
{taxonomy_path}
Y
{model_path}
1
9
EOF
"""

## Using the ! operator to run the command
!echo "Running ilab config init"
!{command}

### 1.2.2 Display the config.yaml file
We examine the base configuration for identifying parameters for changing in the next step.

In [ ]:
##to copy config.yaml to local directory
!cp /root/.config/instructlab/config.yaml .
!cat config.yaml

### 1.2.3 Customize LLM Models and copy to notebook for use

This cell changes the models to use for the generate stage. The mistral model as the teacher model in the generate step and as the student model to be trained.

If you want to customize other models for generation or the training phase, you would specify the models in this step.

This step specifies that the models to be used will be from this notebook's models directory.

In [ ]:
##Use ruamel.yaml to load the yaml file to preserve comments
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open('config.yaml', 'r') as file:
    config = yaml.load(file)

##Upate to use the same models and just change the directory
teacher_model_path = "models/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
base_model_path = "models/instructlab/granite-7b-lab"
##judge_model_path = "models/prometheus-eval/prometheus-8x7b-v2.0"

##config['evaluate']['mt_bench']['judge_model'] = judge_model_path
##config['evaluate']['mt_bench_branch']['judge_model'] = judge_model_path
config['generate']['model'] = teacher_model_path
config['generate']['teacher']['model_path']= teacher_model_path
##config['train']['phased_mt_bench_judge']=judge_model_path

## Save the updated config.yaml file
yaml.default_flow_style=False
with open('config.yaml', 'w') as file:
    yaml.dump(config, file)

##copy the config file to the .config/instructlab/ where it is used by InstructLab
!cp config.yaml {base_dir}.config/instructlab/

print("Updated config.yaml successfully.\n")
!cat config.yaml

<a id="IL1_down"></a>
## Step 1.3 Download Models
The models that will be used in the InstructLab processing are downloaded in this step. Additional steps can be added if other models are used in processing.

### 1.3.1 Download the merlinite and mistral-7b-instruct-v0.2.Q4_K_M models

The merlinite model will be used as the teacher model for the simple pipeline in the [Training with InstructLab](#IL2_0) notebook.

The mistral-7b-instruct-v0.2.Q4_K_M model will be used as the teacher model for the full pipeline in the same notebook.

The granite07b-lab.gguf model is a quantized version oft eh granite-7b-lab model.

In [ ]:
from google.colab import userdata
hf_token=userdata.get('hf_token')
models_dir="models"
!ilab model download --hf-token {hf_token} --model-dir {models_dir}

### 1.3.2 Optionally Download the granite 7b safe tensors model

In [ ]:
##!ilab model download --repository instructlab/granite-7b-lab --hf-token {hf_token} --model-dir {models_dir}

### 1.3.3 Optionally download the prometheus-8x7b-v2.0 model
The *prometheus-8x7b-v2.0* model is used as a judege model for multi-phase training and benchmark evaluation. This model is not required for simple or full training.

In [ ]:
##!ilab model download --repository prometheus-eval/prometheus-8x7b-v2.0 --hf-token {hf_token} --model-dir {models_dir}

<a id="IL2_0"></a>
# Training with InstructLab


This notebook demonstrates InstructLab, a model-agnostic open source AI project that facilitates contributions to Large Language Models (LLMs).

This notebook is part of a sequential notebook set. Before using this notebook, please ensure that you have run the first notebook in this  section: [Configuring InstructLab](#IL1_0).

In this notebook, we will demonstrate the following:
- Querying the LLM before
- Creating a question and answer data file
- Generating synthetic data for training
- Training the LLM with the generated data

**Notes:** This notebook must be run with a GPU. If you are not running with a GPU, please select File->Hub Control Panel->Stop My Server, then Start My Server and the select GPU Session

* [Step 1. Import Libraries and Check Configuration](#IL2_import)
* [Step 2. Specify the Data for this Run](#IL2_data)
* [Step 3. Create the Taxonomy Data Repository](#IL2_taxonomy)
* [Step 4. Generate Synthetic Data](#IL2_generate)
* [Step 5. Train Model](#IL2_train)

<a id="IL2_import"></a>
## Step 2.1 Import Libraries and Check Configuration

### 2.1.1. Imports and configuration


In [ ]:
## standard imports
import subprocess
import json

os.environ['NUMEXPR_MAX_THREADS'] = '64'
il_data_path= '/home/jovyan/.local/share/instructlab/datasets/'
with open('config.json', 'r') as f:
    jsonData = json.load(f)
with open('instructlab.json', 'r') as f:
    jsonState = json.load(f)

### 2.1.2. Optionally test the base model before adding data

At this point you may wish to run an InstructLab server and run queries against the base model.

This may be useful if you are working with new content and want to query the base model to ascertain the responses before InstructLab training.

After training, the [Inferencing with InstructLab](./02_inferencing_with_InstructLab.ipynb) notebook allows you to ask questions to both the base and InstructLab trained models and compare answers.

In [ ]:
##!ilab model serve --model-path models/merlinite-7b-lab-Q4_K_M.gguf

<a id="IL2_data"></a>
## Step 2.2 Specify the Data for this Run

We've provided question-and-answer files for these datasets: "2024 Oscar Awards Ceremony" and "Quantum Roadmap and Patterns" and "Artificial Intelligence Agents". Feel free to choose one of these datasets, or select your own custom dataset in the cell below.

### 2.2.1 Optionally, Create your own data set for InstructLab

Follow these steps to add your own dataset:
1. Create your own **qna.yaml** file based on the example qna.yaml files provided in the /data/oscars, /data/quantum and /data/agentic_ai directories. Additional guidance on creating a properly formatted QNA.yaml file is found on the [InstructLab taxonomy readme](https://github.com/instructlab/taxonomy).
1. Add your **qna.yaml** and sample **questions.txt** files to the **/data/your_content_1** folder or the **/data/your_content_2** folder.
1. Right click on the **config.json** file and select Open With->Editort. Specify the **qna_location** where your data resides within the Dewy Decimal classification system. Close and save the **config.json** file.


In [ ]:
data_set = widgets.ToggleButtons(
    options=['2024 Oscars', 'Quantum', 'Agentic AI', 'Your Content 1', 'Your Content 2'],
    tooltips=['2024 Oscar Awards Ceremony', 'Quantum Roadmap and Patterns', 'Artificial Intelligence Agents', 'Your own uploaded content dataset 1', 'Your own uploaded content dataset 2'],
    description='Dataset:', disabled=False, button_style='', style={"button_width": "auto"}
)
print("\nSelect the QNA dataset to add:")
display(data_set)

print("After choosing your dataset for this run, please select and run the following cell")

In [ ]:
print("Step 2.2 Choose the Dataset for this Run")

if data_set.value=='2024 Oscars':
    use_case="oscars"
elif data_set.value=='Quantum':
    use_case="quantum"
elif data_set.value=='Agentic AI':
    use_case="agentic_ai"
elif data_set.value=='Your Content 1':
    use_case="your_content_1"
elif data_set.value=='Your Content 2':
    use_case="your_content_2"
else:
    use_case="undefined"

if use_case=="undefined":
    print("ERROR: Undefined data set: " + data_set.value + " data")
else:
    jsonState["last_use_case"]=data_set.value
    with open('instructlab.json', 'w') as f:
            json.dump(jsonState, f, indent=4)
    qna_file="data/" + use_case + "/qna.yaml"
    qna_location=jsonData["use_cases"][use_case]["qna_location"]

    print("Using " + data_set.value + " data")

<a id="IL2_taxonomy"></a>
## Step 2.3 Create the Taxonomy Data Repository
### 2.3.1 Delete the prior repository and clone the empty taxonomy repository
We start with an empty repository.

In [ ]:
shell_command1 = f"rm -rf taxonomy"
taxonomy_repo=jsonData["taxonomy_repo"]
shell_command2 = f"git clone {taxonomy_repo}"

print("Step 2.3 Create the Taxonomy data Repository")
print("2.3.1 Delete the prior repository and clone the empty taxonomy repository")
!{shell_command1}
!{shell_command2}

### 2.3.2 View the beginning of the QNA file

In [ ]:
def print_file_top():
    print_lines=40
    with open(qna_file, 'r') as input_file:
        for line_number, line in enumerate(input_file):
            if line_number > print_lines:  # line_number starts at 0.
                break
            print(line, end="")

print("2.3.2 Show QNA file")
print_file_top()

### 2.3.3 Place the QNA file in the proper taxonomy directory

In [ ]:
##Should produce !mkdir -p ./taxonomy/knowledge/textbooks/culture/movies/awards/oscars
shell_command1 = f"mkdir -p ./taxonomy/{qna_location}"
shell_command2 = f"cp ./{qna_file} ./taxonomy/{qna_location}/qna.yaml"

print("2.3.3 Place the QNA file in the proper taxonomy directory")
print("Place QNA file in taxononmy as: /taxonomy/"+qna_location+"/qna.yaml")
!{shell_command1}
!{shell_command2}

### 2.3.4 Verify the taxonomy
We run the ilab taxonomy diff command to verify

In [ ]:
print("Verify the taxonomy")
!ilab taxonomy diff

<a id="IL2_generate"></a>
## Step 2.4. Generate Synthetic Data

This step will produce synthetic training data from the provided repository in the form of question and answer pairs. To generate synthetic data, InstructLab uses s large teacher model, such as Mixtral 8x7B, to create synthetic training data about the manually created data to train a small student model, such as the Merlinite 7B or Granite 7B models.

You can skip this step and use the previously generated 500 synthetic samples found in the *generated* folder.

### 2.4.1 Set data generation parameters

#### Select pipeline

InstructLab has three primary pipelines that can be used: simple, full and acellerated:
- The **simple pipeline** runs fast and can be used for initial model and data testing.
- The **full pipeline** runs all of the InstrctLab steps and takes more time but produces a better tuned model.
- The **accelerated pipeline** runs the full pipeline processing using a GPU, so it produces a similarly tuned model in a shorter time.

**Note:** If you are running with a new or modifed dataset, you may want to use the **Simple pipeline** for the first run to verify the configuration

#### Sepect number of samples to generate

Data generation takes 19 minutes for generating 15 synthetic data samples. You may wish to generate a small number on your first run to verify the QNA dataset format.

To produce **sufficient synthetic data** to focus training on the new material, **about 30 synthetic questions and answer pairs need to be generated** for each question and answer pair provided. This will require a proportionally longer time to generate, but will provide better training.

Before following these instructions, ensure the existing model you are adding skills or knowledge to is still running. Alternatively, ilab data generate can start a server for you if you provide a fully qualified model path via --model.

To generate a synthetic dataset based on your newly added knowledge or skill set in taxonomy repository, run the following command:

    ilab data generate

#### **Simple Pipeline**

The Simple Pipeline works solely with Merlinite 7b Lab as the teacher model. The Simple Pipeline is called without GPU acceleration as follows:

    ilab data generate --pipeline simple

#### **Full Pipeline**

The Full Pipeline runs the full processing with a GPU. Currently, the Full Pipeline only supports the Mixtral and Mistral Instruct Family models as the teacher model.  This is due to only supporting specific model prompt templates.

Using a non-default model such as Mixtral-8x7B-Instruct-v0.1) to generate data with the Full Pipeline:

    ilab data generate --model ~/.cache/instructlab/models/mistralai/mixtral-8x7b-instruct-v0.1 --pipeline full --gpus 4

**Note** Synthetic Data Generation can take from 15 minutes to 1+ hours to complete, depending on your computing resources.

In [ ]:
pipe2 = widgets.ToggleButtons(
    options=['Simple', 'Full with GPU', 'Demo (Use prior data)'],
    tooltips=['Ilab Simple Pipeline', 'Full Pipe running on a GPU', 'Demo Run with previously created data'],
    description='Processing:', disabled=False, button_style='', style={"button_width": "auto"}
)
instr=widgets.ToggleButtons(
    options=['15', '50', '200', '450 (default)', '1000'],
    description='# of QNAs:',
    disabled=False,
    button_style='',
    style={"button_width": "auto"}
)

print("Select Pipeline to use")
display(pipe2)
display(instr)

print("After making your selections for data generation, please select and run the following cell")

### 2.4.2 Run data generation
Data generation takes 19 minutes for generating 15 synthetic data samples and takes longer to generate more samples.

In [ ]:
directory = "data/"+ use_case+"/ilab_generated/"
if pipe2.value!='Demo (Use prior data)':
    if instr.value == '15':
        sdg_factor="--sdg-scale-factor 1"
    elif instr.value == '50':
        sdg_factor="--sdg-scale-factor 3"
    elif instr.value == '200':
        sdg_factor="--sdg-scale-factor 13"
    elif instr.value == '450 (default)':
        sdg_factor=""
    else:
        sdg_factor="--sdg-scale-factor 67"
    # 'Fast (Simple)', 'Full with CPU'
    if pipe2.value == 'Simple':
        pipeline = 'simple'
        model = 'models/merlinite-7b-lab-Q4_K_M.gguf'
        gpus = '--gpus 1'
    elif pipe2.value == 'Full with GPU':
        pipeline = 'full'
        model = 'models/ibm-granite/granite-embedding-125m-english'
        gpus = '--gpus 1'
    else:
        print("ERROR: Undefined pipeline")

    #Remove old data so there is only one test_merlinite and train_merlinite after generation
    !rm -rf /home/jovyan/.local/share/instructlab/datasets/*
    #shell_command = f"ilab --verbose data generate --model {model} --num-cpus 10 {gpus} {sdg_factor} --taxonomy-path taxonomy --pipeline {pipeline} --max-num-tokens 512"
    shell_command = f"ilab data generate --model {model} --num-cpus 10 {gpus} {sdg_factor} --taxonomy-path taxonomy --pipeline {pipeline} --max-num-tokens 512"

    print("Generating data")
    print("Running: !"+shell_command)
    !{shell_command}

    #Rename results to  test_gen.jsonl and train_gen.jsonl and move to local data directory
    if not os.path.exists(directory):
        print("Create directory: " + directory)
        !mkdir {directory}
    file_cnt=0
    try:
        for dirname in os.listdir(il_data_path):
            date_path=il_data_path+'/'+ dirname + '/'
            for filename in os.listdir(date_path):
                if filename[:6]=='train_':
                    train_name= 'train_gen.jsonl'
                    print('Renaming '+ filename+ ' to ' + train_name)
                    !mv {date_path+filename} {directory+train_name}
                    file_cnt+=1
                elif filename[:5]=='test_':
                    test_name= 'test_gen.jsonl'
                    print('Renaming '+ filename+ ' to ' + test_name)
                    !mv {date_path+filename} {directory+test_name}
                    file_cnt+=1
        if file_cnt < 2:
            print("ERROR: train_gen.jsonl and/or test.jsonl not created")
        elif os.path.getsize(directory+train_name) == 0:
            print("ERROR: train_gen.jsonl file is empty")
        elif os.path.getsize(directory+test_name) == 0:
            print("ERROR: test_gen.jsonl file is empty")
        else:
            print("Training and test files successfully created in: " + directory)
    except:
        print("Error running ilab generate, no synthetic data generated")
else:
    print("Using previously generated data")

### 2.4.3 Show examples of generated data

In [ ]:
print("2.4.3 Show examples of generated data")

for filename in os.listdir(directory):
    if filename[:9]=='train_gen':
        with open(directory+filename, 'r') as syn_file:
            cnt=0
            for line_number, line in enumerate(syn_file):
                if cnt >= 8:
                    break
                jsonLine= json.loads(line)
                syn_user=jsonLine["user"]
                syn_assist=jsonLine["assistant"]
                #Remove "Answer:" and "Response:" from anawers for displaying
                if syn_assist[:8]=="Answer: ":
                    syn_assist=syn_assist[8:]
                cnt+=1
                print("\nQuestion: "+syn_user+"\nAnswer: "+syn_assist)

<a id="IL2_train"></a>
## Step 2.5 Train the Model

### 2.5.1 Select the model training pipeline

InstructLab has three primary model training pipelines: simple, full (default), and accelerated. For all of the models, the training time can be limited by adjusting the num_epoch paramater. The maximum number of epochs for running the InstructLab end-to-end workflow is 10.

#### **Simple pipeline**

The simple pipeline uses an SFT Trainer on Linux and MLX on MacOS. This type of training takes roughly an hour and produces the lowest fidelity model but should indicate if your data is being picked up by the training process. The simple pipeline only works with Merlinite 7b Lab as the teacher model. For this Linux system, the trained model is saved in the models directory as ggml-model-f16.gguf.

The command form is:

    ilab model train --pipeline simple

**Note:** This process will take a little while to complete (time can vary based on hardware and output of ilab data generate but on the order of 5 to 15 minutes)

#### **Full pipeline**

The full pipeline uses a custom training loop and data processing functions for the granite family of models. This loop is optimized for CPU and MPS functionality. Please use **--pipeline=full** in combination with **--device=cpu** for this Linus system. For a MacOS system you can use --device=mps (MacOS) or --device=cpu, however, MPS is optimized for better performance on MacOS systems. The full pipeline only works with Mixtral and Mistral Instruct Family models as the teacher model. For the full pipeline, the models are saved in the ~/.local/share/instructlab/checkpoints directory. The instructlab command "ilab model evaluate" can be used to choose the best one.

The command form is:

    ilab model train


**Note:** This process will take a while to complete. If you run for ~8 epochs it will take several hours.

#### **Accelerated pipeline**

The accelerated uses the instructlab-training library which supports GPU accelerated and distributed training. The full loop and data processing functions are either pulled directly from or based off of the work in this library. For the accelerated pipeline, the models are saved in the ~/.local/share/instructlab/checkpoints directory. The instructlab command "ilab model evaluate" can be used to choose the best one. Training is support for GPU acceleration with Nvidia CUDA or AMD ROCm. Please see the GPU acceleration documentation for more details. At present, hardware acceleration requires a data center GPU or high-end consumer GPU with at least 18 GB free memory.

The command form is:

    ilab model train --pipeline accelerated --device cuda --data-path <path-to-sdg-data>

#### **Multiphase**

When running multi phase training evaluation is run on each phase, we will tell you which checkpoint in this folder performs the best. Train the model locally with multi-phase training and GPU acceleration. This results in the following workflow:
1. We train the model on knowledge
1. Evaluate the trained model to find the best checkpoint
1. We train the model on skills
1. We evaluate the model to find the best overall checkpoint

Phase 1 models saved in ~/.local/share/instructlab/phased/phase1/checkpoints (Knowledge training). Phase 2 models saved in ~/.local/share/instructlab/phased/phase2/checkpoints (Skills training). Evaluation is run for phase 2 to identify the best checkpoint.

A multiphase training is of the form:

    ilab model train --strategy lab-multiphase --phased-phase1-data <knowledge train messages jsonl> --phased-phase2-data <skills train messages jsonl> -y

**Note:** This command may take 3 or more hours depending on the size of the data and number of training epochs you run.

#### **Skills only**

Phase 2 models saved in ~/.local/share/instructlab/phased/phase2/checkpoints (Skills training). Evaluation is run for phase 2 to identify the best checkpoint.

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

pipe3 = widgets.ToggleButtons(
    options=['Simple', 'Full with CPU', 'Accelerated GPU', 'Demo (Use prior data)'],
    tooltips=['Ilab Simple Pipeline', 'Full Pipe running only on a CPU', 'Accelerated pipeline run on a GPU', 'Demo Run with previously created data'],
    description='Processing', disabled=False, button_style='', style={"button_width": "auto"}
)
epoch=widgets.ToggleButtons(
    options=['1', '2', '3', '4', '5', '10', '15'],
    description='Epochs:',
    disabled=False, button_style='', style={"button_width": "auto"}
)
it=widgets.ToggleButtons(
    options=['1', '3', '5','10','20','50','100','200'],
    description='Iterations:',
    disabled=False, button_style='', style={"button_width": "auto"}
)

print("Select to Continue or to Train the model")
##pipe3.value=pipe.value
display(pipe3)
display(epoch)
display(it)

print("After choosing your training options, please select and run the following cell")

### 2.5.2 Run the model training

Model training takes 10 minutes for 1 epoch and 1 iteration. This minimal training could be used for testing the generation and training for a new set of data.

To produce a higher quality model, more epochs and iterations are needed for refining the model. This will require a proportionally longer time to train the model.

In [ ]:
data_path="data/"+ use_case+"/ilab_generated/"
train_data=data_path+"train_gen.jsonl"
model_path="models/instructlab/granite-7b-lab"
##model_path='/home/jovyan/.cache/instructlab/models/instructlab/granite-7b-lab'
trained_model_path="data/"+ use_case+"/new_model/"

##'Simple (Fast)', 'Full with CPU', 'Accelerated GPU','DDemo (Use prior data)'
if pipe3.value=='Demo (Use prior data)':
    print("Using previously trained data")
else:
    file_cnt=0
    for filename in os.listdir(data_path):
        if filename[:15]=='train_gen.jsonl': file_cnt+=1
        elif filename[:14]=='test_gen.jsonl': file_cnt+=1
    if file_cnt < 2 or os.path.getsize(directory+train_name) < 5 or os.path.getsize(directory+test_name) < 5:
        print("ERROR: train_gen.jsonl and/or test.jsonl are not present or too small")

    if not os.path.exists(trained_model_path):
        print("Create directory: " + trained_model_path)
        !mkdir {trained_model_path}
    ep=int(epoch.value)
    its=int(it.value)
    if pipe3.value=='Simple':
        print("Train with simple pipeline with a GPU")
        shell_command = f"ilab model train --pipeline simple --model-path {model_path} --data-path {data_path} --device cpu --num-epochs {ep} --iters {its}"
    if pipe3.value=='Full with CPU':
        print("Train with a CPU")
        shell_command = f"ilab model train --pipeline full --model-path {model_path} --data-path {train_data} --device cpu"
    elif pipe3.value=='Accelerated GPU':
        print("Train with a GPU")
        shell_command = f"ilab model train --pipeline accelerated --device cuda --model-path {model_path} --data-path {train_data} --num-epochs {ep} --iters {its}"

    print("Running: !"+shell_command)
    !{shell_command}
    if pipe3.value=='Accelerated GPU' or pipe3.value=='Accelerated GPU with 4b Quantization':
        !ilab model evaluate --benchmark mmlu
    #Move the model to the use_case/new_model directory
    print("Moving the trained model to the directory: "+trained_model_path)
    !mv /home/jovyan/.local/share/instructlab/checkpoints/ggml-model-f16.gguf {trained_model_path}

<a id="IL2_test"></a>
## Step 6 Test the Model
### 2.6.1 Run test on the model to see how it performs

In [ ]:
##!ilab model test

### 2.6.2 Evaluate the model

In [ ]:
##ILAB_MODELS_DIR=$HOME/.local/share/instructlab/models
##shell_command=f"ilab model evaluate --benchmark mmlu --model {ILAB_MODELS_DIR}/instructlab/granite-7b-test"
##!{shell_command}

<a id="I3_0"></a>
# 3. Inferencing with InstructLab

You have now completed InstructLab training. You can now select to run the [Inferencing with InstructLab](./02_inferencing_with_InstructLab.ipynb) notebook to ask questions to both the base and InstructLab trained models and to compare answers.

This notebook is part of a sequential notebook set. Before using this notebook, please ensure that you have reviewed the first and second notebooks in the set:
* <a href="#IL1_0">Configuring InstructLab</a>
* <a href="#IL2_0">Training with InstructLab</a>

The second notebook within this set showcases the generation of synthetic data utilizing InstructLab. It subsequently demonstrates how a large language model (LLM) can be effectively trained on this synthetic dataset. In current notebook, Both the pre-trained LLM and the LLM trained on the generated synthetic data are evaluated against a predefined set of questions to assess their respective performance.

Inferences steps are:

* <a href="#IL3_1">Step 1. Import Libraries </a>
* <a href="#IL3_2">Step 2. Load the Base Model and the InstructLab Trained Model</a>
* <a href="#IL3_3">Step 3. Define a Function to Perform Inference on Base and Trained Models </a>
* <a href="#IL3_4">Step 4. Run Interactive Q&A Session with Base and Trained Models to Evaluate Performance</a>

<a id="IL3_1"></a>
## Step 3.1 Import Libraries

In [ ]:
import shutil
import os
import json
from datasets import load_dataset
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

with open('instructlab.json', 'r') as f:
    jsonState = json.load(f)

print(f"Imports completed")

<a id="IL3_2"></a>
## Step 3.2 Load the Base Model and the InstructLab Trained Model
The data set that was used in that last run of the [Training with InstructLab](#IL2_0) notebook is preselected for inferencing. You may select an alternative dataset if you wish.
You can select the trained model to use for inferencing from:
* **Fine Tuned Model** - A previously trained fine tuned model to demonstrate inferencing.
* **Newly Trained Model** - Your trained model from prior runs of the [Training with InstructLab](#IL2_0) notebook. You can optionally place a model you wish to make comparisons with in the */data/data_set/new_model* directory.


In [ ]:
data_set = widgets.ToggleButtons(
    options=['2024 Oscars', 'Quantum', 'Agentic AI', 'Your Content 1', 'Your Content 2'],
    tooltips=['2024 Oscar Awards Ceremony', 'Quantum Roadmap and Patterns', 'Artificial Intelligence Agents', 'Your own uploaded content dataset 1', 'Your own uploaded content dataset 2'],
    description='Dataset:', disabled=False, button_style='', style={"button_width": "auto"}
)
model = widgets.ToggleButtons(
    options=['Fine Tuned Model', 'Newly Tuned Model'],
    tooltips=['Tested fine tuned model', 'Newly tuned model'],
    description='Model:', disabled=False, button_style='', style={"button_width": "auto"}
)

print("\nPlease select correct document which was used in notebook 01_training_with_InstructLab")
print("\nSelect the content (Last used in training is preselected):")
data_set.value=jsonState["last_use_case"]
display(data_set)
display(model)

print("After choosing your dataset for inferencing, select the following cell and continue running the notebook")

In [ ]:
print("Using Data Set: " + data_set.value)
if data_set.value=='2024 Oscars':
    use_case="oscars"
elif data_set.value=='Quantum':
    use_case="quantum"
elif data_set.value=='Agentic AI':
    use_case="agentic_ai"
elif data_set.value=='Your Content 1':
    use_case="your_content_1"
elif data_set.value=='Your Content 2':
    use_case="your_content_2"
else:
    print("ERROR: Please select correct document which was used in notebook 01_training_with_InstructLab")

if model.value == 'Fine Tuned Model':
    directory="/fine_tuned_models/"+use_case+"/"
else:
    directory="/data/"+ use_case+"/new_model/"

notebook_dir=os.getcwd()
os.chdir('/home/jovyan/')
pwd= os.getcwd()

base_model_path = notebook_dir +"/models/granite-7b-lab-Q4_K_M.gguf"
trained_model_path = notebook_dir + directory+ "ggml-model-f16.gguf"

print("Base model directory: "+base_model_path)
print("Trained model directory: "+trained_model_path)

<a id="IL3_3"></a>
## Step 3.3 Define a Function to Perform Inference on Base and Trained Models

In [ ]:
def model_inference(base_model_path, trained_model_path):
    _DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

    Current conversation:
    Human: {input}
    AI:"""

    base_llm = LlamaCpp(model_path=base_model_path,
                   verbose=False,
                   n_gpu_layers=25,
                   max_tokens=90,
                   temperature=0,
                   top_k=1
                  )
    trained_llm = LlamaCpp(model_path=trained_model_path,
                   verbose=False,
                   n_gpu_layers=25,
                   max_tokens=90,
                   temperature=0,
                   top_k=1
                  )

    PROMPT = PromptTemplate( input_variables=["input"],
                            template=_DEFAULT_TEMPLATE
                            )

    chain1 = PROMPT | base_llm | StrOutputParser()
    chain2 = PROMPT | trained_llm | StrOutputParser()

    while True:
        question = input("Ask me a question (type 'exit' to end): ")
        if question.lower() == 'exit':
            print("Exiting this Q&A session.")
            break
        else:
            print("You asked: ", question)
            answer1 = chain1.invoke(question)
            answer1= answer1.split('Human',1)[0]
            print ("Base Model Answer: ",answer1)
            answer2 = chain2.invoke(question)
            answer2= answer2.split('Human',1)[0]
            print ("Trained Model Answer: ",answer2)

print(f"Function to perform inference on LLMs defined")

<a id="IL3_4"></a>
## Step 3.4 Run Interactive Q&A Session with Base and Trained Models to Evaluate Performance

### 3.4.1 Sample questions that can be asked to LLM

The following are sample questions derived from the data used to generate synthetic data, which was then employed to train the language model.

In [ ]:
##Display Sample Questions
with open(notebook_dir+'/data/' + use_case + '/questions.txt') as f:
    for line in f.readlines():
        display(widgets.HTML(Norm+line))

print("Processing may take several minutes on the first run...")
model_inference(base_model_path, trained_model_path)

<a id="IL3_conclusion"></a>
# Conclusion

This notebook demonstrated utilizing InstructLab for introducing datasets, data generation, model training, and model creation. This notebook produced an InstructLab trained model that was used for inferencing.

<a id="IL3_learn"></a>
# Learn More

Proceed to run the [Inferencing with InstructLab](./02_inferencing_with_InstructLab.ipynb) notebook run inferencing on the InstructLab trained model. This will allow you to interact with your model to see how well it performs on queries, both before it was trained and after InstrutLab training

This notebook is based on the InstructLab CLI repository available [here](https://github.com/instructlab/instructlab).

InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081).

Contact us by email to ask questions, discuss potential use cases, or schedule a technical deep dive. The contact email is IBM.Research.JupyterLab@ibm.com.

© 2025 IBM Corporation